In [1]:
from dotenv import load_dotenv

load_dotenv()

True

## Setup Tools


In [2]:
from langchain_mcp_adapters.client import MultiServerMCPClient

# Search Flight Prices tool from the MCP. This tool is provided by Kiwi.com's Travel Server, which is a server that provides access to a variety of travel-related tools and data.
client = MultiServerMCPClient(
    {
        "travel_server": {
                "transport": "streamable_http",
                "url": "https://mcp.kiwi.com"
            }
    }
)

tools = await client.get_tools()

In [3]:
from typing import Dict, Any
from tavily import TavilyClient
from langchain.tools import tool

tavily_client = TavilyClient()

# Web Search Tool
@tool
def web_search(query: str) -> Dict[str, Any]:

    """Search the web for information"""

    return tavily_client.search(query)

In [4]:
from langchain_community.utilities import SQLDatabase

db = SQLDatabase.from_uri("sqlite:///resources/Chinook.db")

# DB Query Tool that searches playlist information in the Chinook database
@tool
def query_playlist_db(query: str) -> str:

    """Query the database for playlist information"""

    try:
        return db.run(query)
    except Exception as e:
        return f"Error querying database: {e}"

## Create State

In [5]:
from langchain.agents import AgentState

# Coordinator Agent can update the state and the sub-agents can retrieve information from the state as needed.
class WeddingState(AgentState):
    origin: str
    destination: str
    guest_count: str
    genre: str

## Create 3 Subagents
---
1. Travel Agent
1. Venue Search Agent
1. Music Playlist Agent


In [9]:
from langchain.agents import create_agent

# Travel agent
travel_agent = create_agent(
    model="gpt-5-nano",
    tools=tools,
    system_prompt="""
    You are a travel agent. Search for flights to the desired destination wedding location.
    You are not allowed to ask any more follow up questions, you must find the best flight options based on the following criteria:
    - Price (lowest, economy class)
    - Duration (shortest)
    - Date (time of year which you believe is best for a wedding at this location)
    To make things easy, only look for one ticket, one way.
    You may need to make multiple searches to iteratively find the best options.
    You will be given no extra information, only the origin and destination. It is your job to think critically about the best options.
    Once you have found the best options, let the user know your shortlist of options.
    """
)

In [10]:
# Venue agent
venue_agent = create_agent(
    model="gpt-5-nano",
    tools=[web_search],
    system_prompt="""
    You are a venue specialist. Search for venues in the desired location, and with the desired capacity.
    You are not allowed to ask any more follow up questions, you must find the best venue options based on the following criteria:
    - Price (lowest)
    - Capacity (exact match)
    - Reviews (highest)
    You may need to make multiple searches to iteratively find the best options.
    """
)

In [11]:
# Playlist agent
playlist_agent = create_agent(
    model="gpt-5-nano",
    tools=[query_playlist_db],
    system_prompt="""
    You are a playlist specialist. Query the sql database and curate the perfect playlist for a wedding given a genre.
    Once you have your playlist, calculate the total duration and cost of the playlist, each song has an associated price.
    If you run into errors when querying the database, try to fix them by making changes to the query.
    Do not come back empty handed, keep trying to query the db until you find a list of songs.
    You may need to make multiple queries to iteratively find the best options.
    """
)

## Main Coordinator


In [12]:
from langchain.tools import ToolRuntime
from langchain.messages import HumanMessage, ToolMessage
from langgraph.types import Command

@tool
async def search_flights(runtime: ToolRuntime) -> str:
    """Travel agent searches for flights to the desired destination wedding location."""
    origin = runtime.state["origin"]
    destination = runtime.state["destination"]
    response = await travel_agent.ainvoke({"messages": [HumanMessage(content=f"Find flights from {origin} to {destination}")]})
    return response['messages'][-1].content

@tool
def search_venues(runtime: ToolRuntime) -> str:
    """Venue agent chooses the best venue for the given location and capacity."""
    destination = runtime.state["destination"]
    capacity = runtime.state["guest_count"]
    query = f"Find wedding venues in {destination} for {capacity} guests"
    response = venue_agent.invoke({"messages": [HumanMessage(content=query)]})
    return response['messages'][-1].content

@tool
def suggest_playlist(runtime: ToolRuntime) -> str:
    """Playlist agent curates the perfect playlist for the given genre."""
    genre = runtime.state["genre"]
    query = f"Find {genre} tracks for wedding playlist"
    response = playlist_agent.invoke({"messages": [HumanMessage(content=query)]})
    return response['messages'][-1].content

@tool
def update_state(origin: str, destination: str, guest_count: str, genre: str, runtime: ToolRuntime) -> str:
    """Update the state when you know all of the values: origin, destination, guest_count, genre"""
    return Command(update={
        "origin": origin, 
        "destination": destination, 
        "guest_count": guest_count, 
        "genre": genre, 
        "messages": [ToolMessage("Successfully updated state", tool_call_id=runtime.tool_call_id)]}
        )


In [13]:
from langchain.agents import create_agent

coordinator = create_agent(
    model="gpt-5-nano",
    tools=[search_flights, search_venues, suggest_playlist, update_state],
    state_schema=WeddingState,
    system_prompt="""
    You are a wedding coordinator. Delegate tasks to your specialists for flights, venues and playlists.
    First find all the information you need to update the state. Once that is done you can delegate the tasks.
    Once you have received their answers, coordinate the perfect wedding for me.
    """
)


## Test
---

This will take a few minutes to run

In [14]:
from langchain.messages import HumanMessage

response = await coordinator.ainvoke(
    {
        "messages": [HumanMessage(content="I'm from London and I'd like a wedding in Paris for 100 guests, jazz-genre")],
    }
)

In [15]:
from pprint import pprint

pprint(response)

{'destination': 'Paris',
 'genre': 'jazz',
 'guest_count': '100',
 'messages': [HumanMessage(content="I'm from London and I'd like a wedding in Paris for 100 guests, jazz-genre", additional_kwargs={}, response_metadata={}, id='0bc97965-0ae1-4b38-824a-ff2f726c750b'),
              AIMessage(content='', additional_kwargs={'refusal': None}, response_metadata={'token_usage': {'completion_tokens': 421, 'prompt_tokens': 293, 'total_tokens': 714, 'completion_tokens_details': {'accepted_prediction_tokens': 0, 'audio_tokens': 0, 'reasoning_tokens': 384, 'rejected_prediction_tokens': 0}, 'prompt_tokens_details': {'audio_tokens': 0, 'cached_tokens': 0}}, 'model_provider': 'openai', 'model_name': 'gpt-5-nano-2025-08-07', 'system_fingerprint': None, 'id': 'chatcmpl-D92twSVwuUSKdD5GyUO3pgOZB39wb', 'service_tier': 'default', 'finish_reason': 'tool_calls', 'logprobs': None}, id='lc_run--019c5ab7-19f9-7502-8825-a5f530141e9a-0', tool_calls=[{'name': 'update_state', 'args': {'origin': 'London', 'destinat

In [16]:
print(response["messages"][-1].content)

Fantastic — I’ve pulled together initial options for flights, venues, and a starter jazz playlist to set the vibe. I’ve also delegated tasks to the specialists for flights, venues, and playlists so we can move quickly once you decide on a direction.

What I found so far

Flights (London to Paris, around a May wedding window)
- Cheapest one-way options (economy)
  - STN -> CDG: 15 May 2026, 17:00–19:10 (1h10m), ~€53
  - SEN -> CDG: 14 May 2026, 10:10–12:20 (1h10m), ~€59
  - LTN -> CDG: 19 May 2026, 18:10–19:30 (1h20m), ~€57
  - LTN -> CDG: 20 May 2026, 16:10–18:30 (1h20m), ~€58
  - LHR -> ORY: 20 May 2026, 15:00–17:35 (1h35m), ~€58
- Notes: These are direct, one-way economy options from various London airports to CDG or ORY. Availability can change quickly. If you want to explore group-block pricing or specific flight times for guests, I can have the flights team run a dedicated group-rate search.

Venues (Paris region, capacity ~100)
- 1) Rotonde du Glacier, inside Opéra Garnier
  - Ca

link to trace: https://smith.langchain.com/public/7b5fe668-d3e3-4af4-b513-a8cacc0c9e84/r